In [1]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=1)
scipy.__version__

[KeOps] Warning : omp.h header is not in the path, disabling OpenMP. To fix this, you can set the environment
                  variable OMP_PATH to the location of the header before importing keopscore or pykeops,
                  e.g. using os.environ: import os; os.environ['OMP_PATH'] = '/path/to/omp/header'
[KeOps] Warning : Cuda libraries were not detected on the system or could not be loaded ; using cpu only mode
CPU times: user 4.2 s, sys: 1.65 s, total: 5.85 s
Wall time: 16.7 s


'1.15.0'

In [2]:
%%time

acqf = qNoisyExpectedImprovement(model, train_X)

# eval and maximise acq functions
with cProfile.Profile() as pr:
    optimize_acqf(
        acq_function=acqf,
        bounds=bounds_norm,
        q=5,
        num_restarts=5,
        raw_samples=5,
        sequential=False,
    )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__

X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torc

/Users/saitcakmak/botorch/botorch/acquisition/monte_carlo.py:496: NumericsWarning: qNoisyExpectedImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qNoisyExpectedImprovement 	 --> 	 qLogNoisyExpectedImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torch.Size([5, 5, 2]), X.dtype=torch.float64, X.stride()=(10, 2, 1)
X.shape=torc

'1.15.0'

In [3]:
%%time
with cProfile.Profile() as pr:
    for _ in range(100):
        test_X = torch.rand(20, 5, 2, requires_grad=True)
        acqf(test_X).sum().backward()

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats(.1)

scipy.__version__

         195202 function calls (187502 primitive calls) in 0.282 seconds

   Ordered by: cumulative time
   List reduced from 297 to 30 due to restriction <0.1>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  200/100    0.000    0.000    0.178    0.002 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/torch/nn/modules/module.py:1732(_wrapped_call_impl)
  200/100    0.001    0.000    0.177    0.002 /opt/anaconda3/envs/botorch/lib/python3.10/site-packages/torch/nn/modules/module.py:1740(_call_impl)
      100    0.000    0.000    0.177    0.002 /Users/saitcakmak/botorch/botorch/utils/transforms.py:340(decorated)
      100    0.000    0.000    0.177    0.002 /Users/saitcakmak/botorch/botorch/utils/transforms.py:278(decorated)
      100    0.005    0.000    0.175    0.002 /Users/saitcakmak/botorch/botorch/acquisition/monte_carlo.py:254(forward)
      100    0.001    0.000    0.170    0.002 /Users/saitcakmak/botorch/botorch/acquisition/monte_carlo.py:277(_no

'1.15.0'